In [1]:
import random
mapo = {
    0.1: list(range(0,11)),
    0.2: list(range(0,11)),
    0.3: list(range(0,11)),
    0.4: list(range(0,11)),
    0.5: list(range(0,11)),
    0.6: list(range(0,11)),
    0.7: list(range(0,11)),
    0.8: list(range(0,11)),
    0.9: list(range(0,11))
}

In [2]:
random.sample(list(mapo), 9)

[0.9, 0.8, 0.6, 0.4, 0.7, 0.5, 0.1, 0.3, 0.2]

In [3]:
mapo.values()

dict_values([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]])